In [1]:
%load_ext autoreload
%autoreload 2
import os

os.environ["WANDB_SILENT"] = "true"

In [2]:
import argparse
import os
import statistics
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import yaml
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm

import wandb
from src import BertClassifier
from src import datasets as data_utils
from src import influence as inf_utils
from src import train_utils, utils
from src.datasets import create_loo_dataset, create_test_sst2, create_train_sst2

device = utils.get_device()

config = utils.load_config(
    "model_params/bert_classifier.yaml", epochs=5, num_training_examples=1000
)

# Create datasets
train_dataset = create_train_sst2(
    num_samples=config["num_training_examples"],
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
    device=device,
)

test_dataset = create_test_sst2(
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
    device=device,
)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 12040.14it/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel:

## Train Model on Full Data

In [3]:
full_model, fdf, full_test_loss, full_test_acc = train_utils.train_bert_model(
    train_dataset, test_dataset, config
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 30.24batch/s]


In [4]:
TEST_GUID = 716

fdf[fdf.test_guid == TEST_GUID]

,test_guid,logits,pred,label,loss
716,716,"[0.69762474, -0.95851254]",0,0,0.174688


In [5]:
np.argsort(fdf.loss)[300]

257

In [6]:
fdf[fdf.test_guid == 716]

,test_guid,logits,pred,label,loss
716,716,"[0.69762474, -0.95851254]",0,0,0.174688


## Compute Loss Influence

In [7]:
param_infl = list(full_model.classifier.parameters())
infl = inf_utils.compute_influence(
    full_model=full_model,
    test_guid=TEST_GUID,
    param_influence=param_infl,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    lissa_r=2,
    lissa_depth=1,
    damping=5e-3,
    scale=100,
    # training_indices=list(range(15)),
)

LiSSA reps: 2 and num_iterations: 1000
Recursion at depth 0: norm is 4.655922
Recursion at depth 200: norm is 127.890549
Recursion at depth 400: norm is 165.555710
Recursion at depth 600: norm is 177.680008
Recursion at depth 800: norm is 182.073212
Recursion at depth 999: norm is 183.608948
Recursion at depth 0: norm is 4.609522
Recursion at depth 200: norm is 128.841476
Recursion at depth 400: norm is 166.395264
Recursion at depth 600: norm is 178.216537
Recursion at depth 800: norm is 181.195511
Recursion at depth 999: norm is 182.774078


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 138.61it/s]


## Compute LOO Chart

In [8]:
def centered_percentile_idxs(infl, remove_length):
    half = int(len(infl) / 2)
    start_index = max(0, half - int(remove_length / 2))
    end_index = start_index + remove_length
    return np.argsort(infl)[start_index:end_index]

In [12]:
def compute_loo_sweep(test_guid: int) -> pd.DataFrame:
    loo_dfs = []
    # np.arange(0.05, 0.8, 0.05)
    for remove_pct in [0.1]:
        remove_length = int(remove_pct * len(train_dataset))

        # Remove random indices
        remove_idxs = np.random.randint(
            low=0, high=len(train_dataset), size=remove_length
        )
        loo_dataset = create_loo_dataset(train_dataset, remove_idxs)
        _, rdf, rad_test_loss, rand_test_acc = train_utils.train_bert_model(
            loo_dataset,
            test_dataset,
            config,
        )
        rdf["type"] = "rand"

        # Remove top influence score
        remove_idxs = np.argsort(-infl)[:remove_length]
        loo_dataset = create_loo_dataset(train_dataset, remove_idxs)
        _, tdf, rad_test_loss, rand_test_acc = train_utils.train_bert_model(
            loo_dataset, test_dataset, config
        )
        tdf["type"] = "top"

        # Remove bottom influence score
        remove_idxs = np.argsort(infl)[:remove_length]
        loo_dataset = create_loo_dataset(train_dataset, remove_idxs)
        _, bdf, rad_test_loss, rand_test_acc = train_utils.train_bert_model(
            loo_dataset, test_dataset, config
        )
        bdf["type"] = "bot"

        # Remove near 0 influence score
        remove_idxs = centered_percentile_idxs(infl, remove_length)
        loo_dataset = create_loo_dataset(train_dataset, remove_idxs)
        _, zdf, rad_test_loss, rand_test_acc = train_utils.train_bert_model(
            loo_dataset, test_dataset, config
        )
        zdf["type"] = "zero"

        df = pd.concat([rdf, tdf, bdf, zdf], axis=0)
        df["remove_pct"] = remove_pct

        loo_dfs.append(df)
    return pd.concat(loo_dfs)

In [13]:
df = compute_loo_sweep(716)
# df.to_csv('loo_dfs_0.csv', index=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:01<00:00, 28.80batch/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel:

In [14]:
df[(df.type == "rand") & (df.test_guid == TEST_GUID)]

,test_guid,logits,pred,label,loss,type,remove_pct
716,716,"[0.9657729, -1.2266618]",0,0,0.105841,rand,0.1


In [15]:
df[(df.type == "top") & (df.test_guid == TEST_GUID)]

,test_guid,logits,pred,label,loss,type,remove_pct
716,716,"[2.7790165, -3.0399067]",0,0,0.002966,top,0.1


In [16]:
df[(df.type == "bot") & (df.test_guid == TEST_GUID)]

,test_guid,logits,pred,label,loss,type,remove_pct
716,716,"[-0.7447713, 0.48388174]",1,0,1.485376,bot,0.1


In [17]:
df[(df.type == "zero") & (df.test_guid == TEST_GUID)]

,test_guid,logits,pred,label,loss,type,remove_pct
716,716,"[0.6881714, -0.94906056]",0,0,0.177743,zero,0.1


### Plot

In [ ]:
TEST_GUID = 716
base_line_loss = fdf[fdf.test_guid == TEST_GUID].loss.squeeze()

rand = df[(df.type == "rand") & (df.test_guid == TEST_GUID)]
rand = rand[["remove_pct", "loss"]]

zero = df[(df.type == "zero") & (df.test_guid == TEST_GUID)]
zero = zero[["remove_pct", "loss"]]

top = df[(df.type == "top") & (df.test_guid == TEST_GUID)]
top = top[["remove_pct", "loss"]]

bot = df[(df.type == "bot") & (df.test_guid == TEST_GUID)]
bot = bot[["remove_pct", "loss"]]

In [ ]:
base_line_loss

In [ ]:
from matplotlib import pyplot as plt

plt.plot(rand.remove_pct, rand.loss, "g-")
plt.plot(zero.remove_pct, zero.loss, "g--")

# plt.plot(top.remove_pct, top.loss, 'r-')
plt.plot(bot.remove_pct, bot.loss, "b-")

In [ ]:
rand

## Remove Random 10%

In [ ]:
remove_pct = 0.1


remove_length = int(remove_pct * len(train_dataset))


remove_idxs = np.random.randint(low=0, high=max_idx, size=num_indices)
loo_dataset = create_loo_dataset(train_dataset, remove_idxs)

rand_model, rdf, rad_test_loss, rand_test_acc = train_model(
    loo_dataset, test_dataset, config
)

In [ ]:
rdf[rdf.test_guid == TEST_GUID]

## Remove Top 10% Influences

In [ ]:
remove_pct = 0.1
remove_length = int(remove_pct * len(train_dataset))

top_indxs = np.argsort(-infl)[:remove_length]
loo_dataset = create_loo_dataset(train_dataset, top_indxs)

t_model, tdf, top_test_loss, top_test_acc = train_model(
    loo_dataset, test_dataset, config
)

In [ ]:
tdf[tdf.test_guid == TEST_GUID]

## Remove Bottom 10% Influences

In [ ]:
remove_pct = 0.1
remove_length = int(remove_pct * len(train_dataset))

top_indxs = np.argsort(infl)[:remove_length]
loo_dataset = create_loo_dataset(train_dataset, top_indxs)

b_model, bdf, bot_test_loss, bot_test_acc = train_model(
    loo_dataset, test_dataset, config
)

In [ ]:
fdf[fdf.test_guid == TEST_GUID]

In [ ]:
rdf[rdf.test_guid == TEST_GUID]

In [ ]:
tdf[tdf.test_guid == TEST_GUID]

In [ ]:
bdf[bdf.test_guid == TEST_GUID]

In [ ]:
import src.datasets as datasets

datasets.get_test_example(TEST_GUID).sentence.squeeze()

In [ ]:
fdf.sort_values("loss", ascending=False)